In [1]:
import pandas as pd

In [2]:
fraud_data=pd.read_csv("Fraud_check.csv")
fraud_data

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO
...,...,...,...,...,...,...
595,YES,Divorced,76340,39492,7,YES
596,YES,Divorced,69967,55369,2,YES
597,NO,Divorced,47334,154058,0,YES
598,YES,Married,98592,180083,17,NO


In [3]:
fraud_data["taxable_income"]="Risky"
fraud_data.loc[fraud_data["Taxable.Income"]>30000,"taxable_income"]="Good"

In [4]:
fraud_data.drop(["Taxable.Income"],inplace=True,axis=1)
fraud_data['taxable_income'].unique()
fraud_data.taxable_income.value_counts()

Good     476
Risky    124
Name: taxable_income, dtype: int64

In [5]:
fraud_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Undergrad        600 non-null    object
 1   Marital.Status   600 non-null    object
 2   City.Population  600 non-null    int64 
 3   Work.Experience  600 non-null    int64 
 4   Urban            600 non-null    object
 5   taxable_income   600 non-null    object
dtypes: int64(2), object(4)
memory usage: 28.2+ KB


In [6]:
fraud_data = pd.get_dummies(fraud_data, columns = ["Undergrad", "Marital.Status", "Urban"], drop_first = True)

In [7]:
fraud_data.head()

,City.Population,Work.Experience,taxable_income,Undergrad_YES,Marital.Status_Married,Marital.Status_Single,Urban_YES
0,50047,10,Good,0,0,1,1
1,134075,18,Good,1,0,0,1
2,160205,30,Good,0,1,0,1
3,193264,15,Good,1,0,1,1
4,27533,28,Good,0,1,0,0


In [10]:
predictors = fraud_data.loc[:, fraud_data.columns!="taxable_income"]
type(predictors)

pandas.core.frame.DataFrame

In [11]:
predictors

,City.Population,Work.Experience,Undergrad_YES,Marital.Status_Married,Marital.Status_Single,Urban_YES
0,50047,10,0,0,1,1
1,134075,18,1,0,0,1
2,160205,30,0,1,0,1
3,193264,15,1,0,1,1
4,27533,28,0,1,0,0
...,...,...,...,...,...,...
595,39492,7,1,0,0,1
596,55369,2,1,0,0,1
597,154058,0,0,0,0,1
598,180083,17,1,1,0,0


In [12]:
import sklearn
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [13]:
predictors

,City.Population,Work.Experience,Undergrad_YES,Marital.Status_Married,Marital.Status_Single,Urban_YES
0,50047,10,0,0,1,1
1,134075,18,1,0,0,1
2,160205,30,0,1,0,1
3,193264,15,1,0,1,1
4,27533,28,0,1,0,0
...,...,...,...,...,...,...
595,39492,7,1,0,0,1
596,55369,2,1,0,0,1
597,154058,0,0,0,0,1
598,180083,17,1,1,0,0


In [17]:
target = fraud_data["taxable_income"]
type(target)

pandas.core.series.Series

In [18]:
target

0      Good
1      Good
2      Good
3      Good
4      Good
       ... 
595    Good
596    Good
597    Good
598    Good
599    Good
Name: taxable_income, Length: 600, dtype: object

In [19]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(predictors, target, test_size = 0.2, random_state=0)

In [20]:
from sklearn.ensemble import RandomForestClassifier

In [21]:
rf_clf = RandomForestClassifier(n_estimators=500, n_jobs=1, random_state=42)

In [22]:
rf_clf.fit(x_train, y_train)

RandomForestClassifier(n_estimators=500, n_jobs=1, random_state=42)

In [23]:
from sklearn.metrics import accuracy_score, confusion_matrix
confusion_matrix(y_test, rf_clf.predict(x_test))
accuracy_score(y_test, rf_clf.predict(x_test))

0.75

In [24]:
from sklearn.model_selection import GridSearchCV

In [25]:
rf_clf_grid = RandomForestClassifier(n_estimators=500, n_jobs=1, random_state=42)

In [26]:
param_grid = {"max_features": [4, 5, 6, 7, 8, 9, 10], "min_samples_split": [2, 3, 10]}

In [27]:
grid_search = GridSearchCV(rf_clf_grid, param_grid, n_jobs = -1, cv = 5, scoring = 'accuracy')

In [28]:
grid_search.fit(x_train, y_train)

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(n_estimators=500, n_jobs=1,
                                              random_state=42),
             n_jobs=-1,
             param_grid={'max_features': [4, 5, 6, 7, 8, 9, 10],
                         'min_samples_split': [2, 3, 10]},
             scoring='accuracy')

In [29]:
grid_search.best_params_

{'max_features': 6, 'min_samples_split': 10}

In [30]:
cv_rf_clf_grid = grid_search.best_estimator_

In [31]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [32]:
confusion_matrix(y_test, cv_rf_clf_grid.predict(x_test))
accuracy_score(y_test, cv_rf_clf_grid.predict(x_test))

0.775